# If it quacks like a duck...

...And walks like a duck, it is a duck...

We saw an example of Polymorphism before...

In [ ]:
class TrainCar:
    
    def __init__(self, ident):
        self.ident = ident
        
    def volume(self):
        raise NotImplementedError
        
class BoxCar(TrainCar):
    
    def __init__(self, ident, l, w, h):
        super().__init__(ident) # Be DRY
        self.length = l
        self.width = w
        self.height = h
        
    def volume(self):
        return self.width*self.length*self.height

import math
class TankCar(TrainCar):
    
    def __init__(self, ident, l, r):
        super().__init__(ident) # Be DRY
        self.length = l
        self.radius = r
        
    def volume(self):
        return 4*math.pi*self.length*(self.radius**2)
    
b = BoxCar('one', 6, 2, 2)
print(f"Vol for {b.ident}", b.volume())
t = TankCar('two', 6, 1.5)
print(f"Vol for {t.ident}", t.volume())
train = [b, t]
for coach in train:
    print(coach.ident, coach.volume())
print("Train Voume", sum(coach.volume() for coach in train))

Vol for one 24
Vol for two 169.64600329384882
one 24
two 169.64600329384882
Train Voume 193.64600329384882


But python is polymorphic in a much deeper sense...

Let us learn about it by considering sequences in Python

## Sequences in Python

We talked about "listiness" earlier. Lets expand further on it.

Python takes the idea of **protocols** very seriously. 

We have seen that a protocol, the idea that different objects of related types, carry out the same behavior, is key to making software general. For example, all shapes will have an `area` method, all animals will have a `call` method and so on.

But one of the key aspects of python that make it a very natural language to program in is that this idea of polymorphism, this notion of protocols, pervades almost every built in class in the language. These classes may not be related (as different shapes are, and different animals are), but they still have the same methods, and thus can be used in the same ways.

### Sequences

To see this, let us look at many things that look like sequences in Python, but are otherwise different kinds of containers in python. All of these objects are said to follow the **sequence protocol**, which simply says, that one should (a) be able to loop over them, and (b) they should have a `len`gth.

In [ ]:
lst = ['hi', 7, 'c', 2.2]
mystring = 'Hi !'
nums = [1, 4, 7, 9, 12]

`nums` if a list of numbers. You can iterate over them in a list comprehension. So is `lst`, but of different things.

In [ ]:
evens = [e for e in nums if e%2 == 0]

In [ ]:
for element in lst:
    print(element, type(element))

hi <class 'str'>
7 <class 'int'>
c <class 'str'>
2.2 <class 'float'>


But look here, the same loop can be used to iterate over strings.

In [ ]:
for character in mystring:
    print(character)

H
i
 
!


Each of these has a length:

In [ ]:
len(lst)

4

In [ ]:
len(evens)

2

In [ ]:
len(mystring)

4

And you can index into them:

In [ ]:
lst[0], mystring[1], nums[2]

('hi', 'i', 7)

What happens if we look at a dictionary

In [ ]:
d = dict(   
    name = 'Alice',
    age  = 18,
    gender = 'F'
)
print("age", d['age'])
for item in d:
    print(item)

age 18
name
age
gender


We get the keys in some order (order is not guaranteed in dictionaries. Use an `OrderedDict` for that)

In [ ]:
len(d)

3

What about tuples?

In [ ]:
tup = (1, 2, 3)

In [ ]:
for i in tup:
    print(i)

1
2
3


In [ ]:
len(tup)

3

All off lists, tuples, dictionaries, and strings follow the *sequence protocol*. What if you wanted to write your own class that supports the protocol?

### The sequence protocol, formally: Dunder methods

Any class which wants to be a sequence, must support having a length, being iterated over, and being indexable. It turns out that in Python, this is ensured by having 2 special methods defined within the class. These are `__len__`, which calculates the length of the class. When you call `len` on an instance of this class, python will *dispatch* to this **dunder** (or double-underscore) method defined by the class. The second one is `__getitem__(self, i)__`, which tells you whats in the sequence at index i. By having this we can now loop over `i`s until the `len` to iterate over the *sequence*. Here is such an implementation:

In [ ]:
class Vector:
    
    def __init__(self, lst):
        self.storage = lst
        
    def __len__(self):
        return len(self.storage)
    
    def __getitem__(self, i):
        return self.storage[i]



This is not a particularly interesting implementation, it simply *delegates* the length calculation and indexing to the underlying list-based `self.storage` instance variable.

In [ ]:
v = Vector([3, 4, 5, 6])

In [ ]:
v

<__main__.Vector>

In [ ]:
v[1]

4

In [ ]:
for i in v:
    print(i)

3
4
5
6


But as you can see, it does show how easy it isnto behave like a sequence. This type of protocol/polymorphism is called *Duck Typing*, a term coined by Alex Martelli, which roughly means: If it quacks like a duck (sequence) it is a Duck (sequence).

## Printing nicer

In [ ]:
v

<__main__.Vector>

This is not a particularly useful printout. Lets fix this:

In [ ]:
class Vector:
    
    def __init__(self, lst):
        self.storage = lst.copy()
        
    def __len__(self):
        return len(self.storage)
    
    def __getitem__(self, i):
        return self.storage[i]
    
    def __repr__(self):
        return f"Vector({self.storage})"

v = Vector([3, 4, 5, 6])

In [ ]:
v

Vector([3, 4, 5, 6])

Much better! Implementing `__repr__` lets us print our object in a human-readable way. We have used a format string or a f-string to do this. Its a wierd kind of string, with a "f" in front of it. Python keeps this "one letter in front of string" syntax for alternative strings such as f-strings (start with f), byte strings (startwith b), etc. 

In an f-string you can interpolate python variables if you enclose them in braces. Here we'll put in the full contents of self.storage. Terrible if this is a really long list but we'll let it be for now.

## One last thing: operator overloading

How do we make this Vector class useful? Python uses its dunder methods for this as well:

In [ ]:
class Vector:
    
    def __init__(self, lst):
        self.storage = lst
        
    def __len__(self):
        return len(self.storage)
    
    def __getitem__(self, i):
        return self.storage[i]

    def __add__(self, other_vector):
        sumlist = []
        for i, _ in enumerate(other_vector):
            sumlist.append(self.storage[i] + other_vector[i])
        return Vector(sumlist)
    
    def __repr__(self):
        return f"Vector({self.storage})"

In [ ]:
v1 = Vector([4, 2, 7])
v2 = Vector([1, -1, 3])
v1+v2

Vector([5, 1, 10])

This is what happens when we do the addition:

In [ ]:
v1.__add__(v2)

Vector([5, 1, 10])

We can add lists to vectors: why? This is the beauty of Python protocols: lists support iteration and thus we just iterate over them and add one by one.

In [ ]:
v1 + [-1, -1, 3]

Vector([3, 1, 10])

But its not possible to add a list to a vector

In [ ]:
[-1, -1, 3] + v1

TypeError: can only concatenate list (not "Vector") to list

In the Python Data Model, dunder methods starting with `__r `need to be implemented to figure when the new class is on the right side of the operator. Here we define `__radd__`.

Our implementation of `__radd__` simply reverses the direction of addition to take advantage of our working left addition.

We also see an example of Python's error handling , using try and catch. If we get a type error (as in adding an integer) we return `NotImplemented` which allows Python to try right addition, in case the other type implements left addition with something like a vector (this is not true for integers)

In [ ]:
class Vector:
    
    def __init__(self, lst):
        self.storage = lst
        
    def __len__(self):
        return len(self.storage)
    
    def __getitem__(self, i):
        return self.storage[i]

    def __add__(self, other_vector):
        try:
            sumlist = []
            for i, _ in enumerate(other_vector):
                sumlist.append(self.storage[i] + other_vector[i])
            return Vector(sumlist)
        except TypeError:
            return NotImplemented
    
    def __radd__(self, other_vector):
        # turn other + self around
        return self + other_vector
    
    def __repr__(self):
        return f"Vector({self.storage})"

In [ ]:
v1 = Vector([4, 2, 7])
v2 = Vector([1, -1, 3])
v1+v2

Vector([5, 1, 10])

In [ ]:
[-1, -1, 3] + v1

Vector([3, 1, 10])